In [1]:
cd ..

/Users/dorislee/Desktop/Research/covid/covid19-vis


In [2]:
import pandas as pd

In [3]:
%load_ext autoreload
%autoreload 2
from chartlib.covid_chart import CovidChart

In [4]:
df = pd.read_csv("data/quarantine-activity-US-Apr16-long.csv", thousands=',')
df2 = pd.read_csv("data/reopening-activity-US-Jun9.csv",index_col=0)
df2["Effective Date"] = pd.to_datetime(df2["Effective Date"]).dt.strftime("%m-%d-%Y")
# df2.to_csv("data/reopening-activity-US-Jun9.csv",index=None)
print(len(df))
print(len(df2))

607
108


In [5]:
# Generate state population lookup table
quarantine_df_orig = df[df["Coverage.type"]=="State-wide"]
state_populations = quarantine_df_orig[["State","population_size"]].drop_duplicates()
state_populations = state_populations.rename(columns={'State': 'Province_State'})#,"population_size":"state_population_size"})
state_populations.loc[state_populations["Province_State"]=="Puerto Rico","population_size"] = 3725789 # Obtained from https://www.census.gov/quickfacts/PR
state_populations.to_csv("data/state_population_lookup.csv",index=None)

df2 = df2.rename(columns={'Coverage':'Coverage.type'}) 
df2["Coverage.location"] = df2.State # Coerce all the Coverage location to be State-wide

state_populations.index = state_populations.Province_State
state_populations = state_populations.drop(columns="Province_State",index=None)
state_population_lookup = state_populations.to_dict()
state_population_lookup = state_population_lookup["population_size"]

import json
with open('data/state_population_lookup.json', 'w') as outfile:
    json.dump(state_population_lookup, outfile)

In [6]:
df2["Coverage.type"].unique()

array(['State-wide', 'Selected counties (based on specific metrics)',
       'Selected counties', 'Palm Beach ', 'Miami Dade County',
       'Broward County', 'Lake County', 'Marion County', 'Cass County',
       'Rural areas', 'Selected regions',
       'Selected regions (based on specific metrics)',
       'Selected counties (most of state)'], dtype=object)

In [68]:
df2["Effective Date"].max()

'05-27-2020'

In [7]:
# Change all population size to be the state-wide population, if lower than state, automatically assign as 0.5* state population
def assignPopSize(x):
    stateSize = int(state_population_lookup[x["State"]])
    if (x["Coverage.type"]=='State-wide'):
        return stateSize
    else:
        return round(0.5* stateSize)

In [8]:
df2["population_size"] = df2.apply(assignPopSize, axis=1)
# # df2[df2["Coverage.type"]=='State-wide']
# # df2["population_size"] = df2[df2["Coverage.type"]!='State-wide'].apply(lambda x: 0.5*state_population_lookup[x["State"]],axis=1)

In [9]:
combined_df = pd.concat([df,df2])
print (len(combined_df))

715


In [13]:
combined_df.to_csv("data/combined-activity-US-Jun9-with-partial-pop-b4-Apr16.csv")

#### ---------------------------interventionFootprint.py---------------------------

In [15]:
cd chartlib/

/Users/dorislee/Desktop/Research/covid/covid19-vis/chartlib


In [40]:
severityScore = {'e':0.1,'g':0.3, 'c':0.3, 's':0.4,'r':0.6 , 't':0.6, 'n':0.8, 'l':1, 'f':0.5,
                 'a':-0.1, 'd':-0.6, 'k':-0.3, 'h':-1, 'n':-0.8, 'p':-0.5, 'q':-0.6, 'o':-0.8, 'j':-0.3}


import pandas as pd
import numpy as np
from utils import create_lockdown_type, split_into_list, str2emo

quarantine_csv = "../data/combined-activity-US-Jun9-with-partial-pop-b4-Apr16.csv"
quarantine_df = pd.read_csv(quarantine_csv)

quarantine_df = quarantine_df.rename(columns={'Effective.Date' : 'Effective Date', 'State.of.Emergency.Declaration' : 'State of Emergency Declaration', 
                'Travel.Restrictions' : 'Travel Restrictions', 'Shelter.in.place.Order' : 'Shelter-in-place Order', 
                'Gathering.Limitations' : 'Gathering Limitations', 'Banning.Gatherings.of.a.Certain.Size' : 'Banning Gatherings of a Certain Size',
                'K.12.School.Closure' : 'K-12 School Closure', 'Bar.and.Dine.in.Restaurant.Closure' : 'Bar and Dine-in Restaurant Closure',
                'Non.essential.Businesses.Closure' : 'Non-essential Businesses Closure', 'Details..if.any.' : 'Details (if any)',
                'Reference.links' : 'Reference links', 'Face.Covering.Requirements' : 'Face Covering Requirements', 'state_pop_2019' : 'population_size'})
quarantine_df = quarantine_df.drop('Unnamed: 0', axis=1)


groupcol = 'Province_State'

quarantine_df = quarantine_df.rename(columns={'State': 'Province_State', 'Effective Date': 'lockdown_date','Coverage.type':'Coverage'})
quarantine_df = quarantine_df.sort_values('Coverage', ascending=True)
quarantine_df['lockdown_type'] = quarantine_df.apply(lambda x: create_lockdown_type(x, 0), axis=1)
quarantine_df['emoji_string'] = quarantine_df.apply(lambda x: create_lockdown_type(x, 1), axis=1)
quarantine_df['lockdown_type'].replace('', np.nan, inplace=True)

quarantine_df = quarantine_df.dropna(subset=['lockdown_type'])

In [41]:
groupcol = 'Province_State'

quarantine_df = quarantine_df.rename(columns={'State': 'Province_State', 'Effective Date': 'lockdown_date','Coverage.type':'Coverage'})
quarantine_df = quarantine_df.sort_values('Coverage', ascending=True)
quarantine_df['lockdown_type'] = quarantine_df.apply(lambda x: create_lockdown_type(x, 0), axis=1)
quarantine_df['emoji_string'] = quarantine_df.apply(lambda x: create_lockdown_type(x, 1), axis=1)
quarantine_df['lockdown_type'].replace('', np.nan, inplace=True)

quarantine_df = quarantine_df.dropna(subset=['lockdown_type'])

In [42]:
quarantine_df.loc[quarantine_df.lockdown_type=="Regional Border Closure/Visitor Quarantine","emoji_string"]="t"#bugfix
# Breaking up emoji into separate rows for vertical stacking
quarantine_df.emoji_string = quarantine_df.emoji_string.apply(split_into_list)

quarantine_df = quarantine_df.explode(column='emoji_string')
quarantine_df['Coverage'] = quarantine_df.emoji_string.apply(
    lambda x: 'Statewide' if str(x).isupper() else 'Regional'
)
quarantine_df = quarantine_df.sort_values('Coverage', ascending=False)
quarantine_df.emoji_string = quarantine_df.emoji_string.str.lower()
quarantine_df['emoji'] = quarantine_df['emoji_string'].map(str2emo)
quarantine_df['event_index'] = quarantine_df.groupby(['Province_State', 'lockdown_date']).cumcount()


In [43]:
quarantine_cols = ['Province_State', 'Coverage','Coverage.location', 'lockdown_date','population_size','lockdown_type', 'emoji_string', 'emoji',
       'event_index']

print(quarantine_df.columns)
quarantine_df = quarantine_df[quarantine_cols]

# # END modified from _ingest_usa_quarantine_df

quarantine_df_orig = pd.read_csv(quarantine_csv)
quarantine_df_orig = quarantine_df_orig.rename(columns={'Effective.Date' : 'Effective Date', 'State.of.Emergency.Declaration' : 'State of Emergency Declaration', 
                'Travel.Restrictions' : 'Travel Restrictions', 'Shelter.in.place.Order' : 'Shelter-in-place Order', 
                'Gathering.Limitations' : 'Gathering Limitations', 'Banning.Gatherings.of.a.Certain.Size' : 'Banning Gatherings of a Certain Size',
                'K.12.School.Closure' : 'K-12 School Closure', 'Bar.and.Dine.in.Restaurant.Closure' : 'Bar and Dine-in Restaurant Closure',
                'Non.essential.Businesses.Closure' : 'Non-essential Businesses Closure', 'Details..if.any.' : 'Details (if any)',
                'Reference.links' : 'Reference links', 'Face.Covering.Requirements' : 'Face Covering Requirements', 'state_pop_2019' : 'population_size'})
quarantine_df_orig = quarantine_df_orig.drop('Unnamed: 0', axis=1)

# state_populations = quarantine_df_orig[["State","population_size"]].drop_duplicates()
state_populations = state_populations.rename(columns={'State': 'Province_State',"population_size":"state_population_size"})
# state_populations.loc[state_populations["Province_State"]=="Puerto Rico","state_population_size"] = 3725789 # Obtained from https://www.census.gov/quickfacts/PR
# state_populations.state_population_size = state_populations.state_population_size.astype("int")

quarantine_df = quarantine_df.merge(state_populations,on="Province_State")

# # quarantine_df = quarantine_df.dropna(subset=["population_size"])
# quarantine_df.population_size = quarantine_df.population_size.astype("int")

Index(['Province_State', 'Coverage', 'Coverage.location', 'lockdown_date',
       'State of Emergency Declaration', 'Travel Restrictions',
       'Shelter-in-place Order', 'Gathering Limitations',
       'Banning Gatherings of a Certain Size', 'K-12 School Closure',
       'Bar and Dine-in Restaurant Closure',
       'Non-essential Businesses Closure', 'Details (if any)',
       'Reference links', 'Timestamp', 'population_size',
       'population_reference', 'Coverage.county.FIPS', 'Coverage.city.FIPS',
       'population_reference.1', 'Unnamed: 0.1', 'Face Covering Requirements',
       'Details (if any) ', 'lockdown_type', 'emoji_string', 'emoji',
       'event_index'],
      dtype='object')


In [44]:
quarantine_df["severityScore"] = quarantine_df.emoji_string.apply(lambda x: severityScore[x])
quarantine_df.lockdown_date = quarantine_df.lockdown_date.str.replace("/","-")
quarantine_df.lockdown_date = pd.to_datetime(quarantine_df.lockdown_date)
quarantine_df = quarantine_df[['Province_State', 'Coverage', 'Coverage.location', 'lockdown_date',
       'population_size', 'state_population_size', 'severityScore']]

In [80]:

def compute_state_replacements(qdf,state):
    df_state_replacements = []
    qdf = quarantine_df[quarantine_df["Province_State"]==state]
    statewideEvents = qdf[qdf["Coverage"]=="Statewide"]
    relevantCountiesInState= list(set(qdf[qdf["Province_State"]==state]["Coverage.location"]) - set([state]))
    # adding "Others" dummy state populated with statewide population - sum(relevant counties population)
    relevantCountiesInState= list(set(qdf[qdf["Province_State"]==state]["Coverage.location"]) - set([state]))
    relevant_county_population_lookup = qdf[qdf["Coverage.location"].isin(relevantCountiesInState)][["Coverage.location","population_size"]].drop_duplicates()
#     state_population_size = state_populations[state_populations["Province_State"]==state].state_population_size.values[0]
    state_population_size = state_population_lookup[state]
    other_population = state_population_size - relevant_county_population_lookup.population_size.sum()
    qdf.loc[len(qdf)] = [state,"Regional","Others",np.nan,other_population,state_population_size,np.nan]
    relevantCountiesInState  = relevantCountiesInState + ["Others"]
    for statewideEvent in statewideEvents.iterrows(): 
    #     statewideLockdown_date = "2020-03-12"
    #     statewideSeverity = "0.1"
        statewideLockdown_date = statewideEvent[1].lockdown_date
        statewideSeverity = statewideEvent[1].severityScore
        for county in relevantCountiesInState: 
            #print(county)
            #display(qdf[qdf["Coverage.location"]==county])
            try:
                # Catch error since some counties don't show up until a later date, ignore these counties 
                clone = np.repeat(qdf[qdf["Coverage.location"]==county].iloc[0],1)
                clone.lockdown_date = statewideLockdown_date
                clone.severityScore = statewideSeverity
                df_state_replacements.append(clone)
            except (IndexError):
                pass

    #df_state_replacements = pd.concat(df_state_replacements)
    df_state_replacements = pd.DataFrame(df_state_replacements,columns=qdf.columns)
    return df_state_replacements 

def compute_state_intervention_footprint_curve(state):
    qdf = quarantine_df[quarantine_df["Province_State"]==state]
    qdf_additional = compute_state_replacements(qdf,state)
    qdf_result = pd.concat([qdf,qdf_additional])
    # For each date, compute a cumulative max, keeping only the entry with the maximum severity score
    qdf_keep_all=[]
    for ldate in qdf.lockdown_date.unique():
        # ldate = "2020-03-13"
        qdf_before = qdf_result[qdf_result["lockdown_date"]<=ldate]
        # Compute the max severity for that location for the given day
        #qdf_before["severityMax"] = qdf_before.groupby("Coverage.location")["severityScore"].transform(max)
        #qdf_keep = qdf_before[qdf_before["severityScore"] == qdf_before["severityMax"]]
        print (qdf_before)
        qdf_keep = qdf_before
        qdf_keep["dateBefore"] = ldate
        qdf_keep = qdf_keep.drop_duplicates(subset="Coverage.location") # applicable only for the last date when severityMax=1 for multiple lockdown_dates, keep just one so that last entry doesn't exceed 1
        qdf_keep_all.append(qdf_keep)
    #     display(qdf_keep)
    qdf_keep_all = pd.concat(qdf_keep_all)

    # compute the intervention footprint for the kept df
    qdf_keep_all["pctStateAffected"]= qdf_keep_all["population_size"]/qdf_keep_all["state_population_size"]
    qdf_keep_all["interventionFootprint"] = qdf_keep_all["pctStateAffected"]*qdf_keep_all["severityScore"]

    qdf_keep_all = qdf_keep_all[qdf_keep_all["Coverage"]=="Regional"]
    # Sum the intervention footprint based on the date we used for the keep max (note this is based on the lockdown date)
    qdf_keep_all_result = qdf_keep_all.groupby("dateBefore").sum().reset_index()
    return qdf_keep_all_result

In [81]:
df_all = []
for state in quarantine_df.Province_State.unique():
    print (state)
    
    statedf = compute_state_intervention_footprint_curve(state)
    statedf["State"]=state
    df_all.append(statedf)

df_all = pd.concat(df_all)

Wyoming
   Province_State   Coverage Coverage.location lockdown_date  population_size  \
0         Wyoming  Statewide           Wyoming    2020-05-15           578759   
1         Wyoming  Statewide           Wyoming    2020-05-08           578759   
2         Wyoming  Statewide           Wyoming    2020-05-08           578759   
3         Wyoming  Statewide           Wyoming    2020-05-15           578759   
4         Wyoming  Statewide           Wyoming    2020-03-15           578759   
5         Wyoming  Statewide           Wyoming    2020-03-13           578759   
6         Wyoming  Statewide           Wyoming    2020-05-01           578759   
7         Wyoming  Statewide           Wyoming    2020-05-01           578759   
8         Wyoming  Statewide           Wyoming    2020-05-01           578759   
9         Wyoming   Regional             Teton    2020-03-28            23464   
10        Wyoming   Regional           Jackson    2020-03-31            10429   
11        Wyoming   

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

   Province_State   Coverage Coverage.location lockdown_date  population_size  \
23  Massachusetts  Statewide     Massachusetts    2020-03-24          6892503   
24  Massachusetts  Statewide     Massachusetts    2020-03-24          6892503   
25  Massachusetts  Statewide     Massachusetts    2020-03-24          6892503   
26  Massachusetts  Statewide     Massachusetts    2020-03-17          6892503   
27  Massachusetts  Statewide     Massachusetts    2020-03-17          6892503   
28  Massachusetts  Statewide     Massachusetts    2020-03-17          6892503   
29  Massachusetts  Statewide     Massachusetts    2020-03-17          6892503   
30  Massachusetts  Statewide     Massachusetts    2020-03-10          6892503   
35  Massachusetts   Regional    City of Boston    2020-03-13           694583   
36  Massachusetts   Regional         Wellesley    2020-03-13            27982   
37  Massachusetts   Regional        Framingham    2020-03-13            73123   
36  Massachusetts   Regional

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

    Province_State   Coverage Coverage.location lockdown_date  \
71        Oklahoma  Statewide          Oklahoma    2020-03-25   
72        Oklahoma  Statewide          Oklahoma    2020-03-15   
73        Oklahoma  Statewide          Oklahoma    2020-03-24   
74        Oklahoma  Statewide          Oklahoma    2020-03-24   
79        Oklahoma   Regional            Garvin    2020-03-25   
..             ...        ...               ...           ...   
110       Oklahoma   Regional             Texas    2020-03-24   
89        Oklahoma   Regional     City of Tulsa    2020-03-24   
94        Oklahoma   Regional             Mayes    2020-03-24   
106       Oklahoma   Regional            Pawnee    2020-03-24   
66        Oklahoma   Regional            Others    2020-03-24   

     population_size  state_population_size  severityScore  
71           3956971                3956971            0.4  
72           3956971                3956971            0.1  
73           3956971                

...

Illinois
    Province_State   Coverage Coverage.location lockdown_date  \
971       Illinois  Statewide          Illinois    2020-03-16   
972       Illinois  Statewide          Illinois    2020-03-16   
974       Illinois  Statewide          Illinois    2020-03-15   
975       Illinois  Statewide          Illinois    2020-03-09   
7         Illinois   Regional            Others    2020-03-16   
7         Illinois   Regional            Others    2020-03-16   
7         Illinois   Regional            Others    2020-03-15   
7         Illinois   Regional            Others    2020-03-09   

     population_size  state_population_size  severityScore  
971         12671821               12671821            0.3  
972         12671821               12671821            0.3  
974         12671821               12671821            0.6  
975         12671821               12671821            0.1  
7           12671821               12671821            0.3  
7           12671821               1267

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab


     Province_State   Coverage Coverage.location lockdown_date  \
1002       Arkansas  Statewide          Arkansas    2020-03-19   
1003       Arkansas  Statewide          Arkansas    2020-03-17   
1004       Arkansas  Statewide          Arkansas    2020-03-11   
1005       Arkansas  Statewide          Arkansas    2020-05-04   
1006       Arkansas  Statewide          Arkansas    2020-05-04   
1007       Arkansas  Statewide          Arkansas    2020-05-18   
1008       Arkansas  Statewide          Arkansas    2020-05-11   
1009       Arkansas   Regional             Grant    2020-03-12   
1010       Arkansas   Regional         Jefferson    2020-03-12   
1011       Arkansas   Regional           Pulaski    2020-03-12   
1012       Arkansas   Regional            Saline    2020-03-12   
1011       Arkansas   Regional           Pulaski    2020-03-19   
1010       Arkansas   Regional         Jefferson    2020-03-19   
1012       Arkansas   Regional            Saline    2020-03-19   
1009     

West Virginia
     Province_State   Coverage Coverage.location lockdown_date  \
1059  West Virginia  Statewide     West Virginia    2020-03-14   
9     West Virginia   Regional            Others    2020-03-14   

      population_size  state_population_size  severityScore  
1059          1792147                1792147            0.4  
9             1792147                1792147            0.4  
     Province_State   Coverage Coverage.location lockdown_date  \
1059  West Virginia  Statewide     West Virginia    2020-03-14   
1060  West Virginia  Statewide     West Virginia    2020-03-23   
1061  West Virginia  Statewide     West Virginia    2020-03-16   
9     West Virginia   Regional            Others    2020-03-14   
9     West Virginia   Regional            Others    2020-03-23   
9     West Virginia   Regional            Others    2020-03-16   

      population_size  state_population_size  severityScore  
1059          1792147                1792147            0.4  
1060          

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [72]:
df_all.to_csv("../data/interventionFootprintByStateReopeningUpdateJun9.csv")

## Visualization

In [73]:
import pandas as pd
df_all = pd.read_csv("../data/interventionFootprintByStateReopeningUpdateJun9.csv",index_col=0)

In [74]:
df_all["dateBefore"]= pd.to_datetime(df_all.dateBefore)

In [79]:
import altair as alt
chart = alt.Chart(df_all).mark_line().encode(
        x = alt.X("dateBefore"),
        y = alt.Y("interventionFootprint",scale=alt.Scale(0,1) ),
        facet=alt.Facet('State:O', columns=4),
    )
chart.properties(width=150,height=100,title="Step-Function Curve").interactive()

alt.Chart(...)